# Multi-modal eval: Baseline

`Multi-modal slide decks` is a public dataset that contains a dataset of question-answer pairs from slide decks with visual content.

The question-answer pairs are derived from the visual content in the decks, testing the ability of RAG to perform visual reasoning.

As a baseline, we evaluate this dataset using text-based RAG pipeline, below.

This will not reason about visual content and will simply load the text from the slides. 

## Pre-requisites

In [ ]:
# %pip install -U langchain langsmith langchain_benchmarks
# %pip install --quiet chromadb openai pypdf pandas

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
env_vars = ["LANGCHAIN_API_KEY", "OPENAI_API_KEY"]
for var in env_vars:
    if var not in os.environ:
        os.environ[var] = getpass.getpass(prompt=f"Enter your {var}: ")

## Dataset

We can browse the available LangChain benchmark datasets for retrieval.

In [ ]:
from langchain_benchmarks import clone_public_dataset, registry

registry = registry.filter(Type="RetrievalTask")
registry

`Multi-modal slide decks` is the relevant dataset for our task.

In [ ]:
task = registry["Multi-modal slide decks"]
task

Clone the dataset so that it's available in our LangSmith datasets.

In [ ]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Fetch the associated PDFs from remote cache for the dataset so that we can perform ingestion.

In [ ]:
from langchain_benchmarks.rag.tasks.multi_modal_slide_decks import get_file_names

file_names = list(get_file_names())  # PosixPath

## Load

Load and split the files for indexing.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_split(file):
    """
    Load and split PDF files
    :param file: PosixPath path for pdf
    :return: A list of text chunks
    """

    loader = PyPDFLoader(str(file))
    pdf_pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=100, chunk_overlap=50
    )

    # Get chunks
    docs = text_splitter.split_documents(pdf_pages)
    texts = [d.page_content for d in docs]
    print(f"There are {len(texts)} text elements in {file.name}")
    return texts


texts = []
for fi in file_names:
    texts.extend(load_and_split(fi))

## Index

Embed (OpenAIEmbeddings) and store splits in a vectorstore (Chroma).

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore_baseline = Chroma.from_texts(
    texts=texts, collection_name="baseline-multi-modal", embedding=OpenAIEmbeddings()
)

retriever_baseline = vectorstore_baseline.as_retriever()

## RAG

Create a pipeline for retrieval of relevant chunks based on semantic similarity to the input question.

Pass the images to GPT-4 for answer synthesis.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


def rag_chain(retriever):
    """
    RAG pipeline for the indexed presentations
    :param retriever: PosixPath path for pdf
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {
            "context": retriever | (lambda x: "\n\n".join([i.page_content for i in x])),
            "question": RunnablePassthrough(),
        }
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Create RAG chain
chain = rag_chain(retriever_baseline)

## Eval

Run evaluation on our dataset:

* `task.name` is the dataset of QA pairs that we cloned
* `eval_config` specifies the [LangSmith evaluator](https://docs.smith.langchain.com/evaluation/evaluator-implementations#correctness-qa-evaluation) for our dataset, which will use GPT-4 as a grader
* The grader will evaluate the chain-generated answer to each question relative to ground truth

In [ ]:
import uuid

from langchain.smith import RunEvalConfig
from langsmith.client import Client

# Evaluator configuration
client = Client()
eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

# Experiments
chain_map = {
    "baseline": chain,
}

# Run evaluation
run_id = uuid.uuid4().hex[:4]
test_runs = {}
for project_name, chain in chain_map.items():
    test_runs[project_name] = client.run_on_dataset(
        dataset_name=task.name,
        llm_or_chain_factory=lambda: (lambda x: x["Question"]) | chain,
        evaluation=eval_config,
        verbose=True,
        project_name=f"{run_id}-{project_name}",
        project_metadata={"chain": project_name},
    )